# POM Nettoyage de donnees IOS 

## Importatation des Bibliotheques

+   Importatation de Pandas

In [ ]:
import pandas as pd

*   Importatation de Numpy

In [ ]:
import numpy as np

## Imprtation du fichier .csv depuis un url:

* Installation de Google Api Python 
* Installation de Google Authentification
* Installation de Google Api Client

In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from __future__ import print_function
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
# SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/Account_Service.json'
SERVICE_ACCOUNT_FILE = 'Account_Service.json'

creds = None
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1JDvSxF6V-4q1qjMCKomOwgx7TrGKRYw49HpduWJ-eI4'

service = build('sheets','v4', credentials=creds)
# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,range="A1:N3720").execute()
values = result.get('values', [])

## Transformation des resultats en Data Frame

In [ ]:
colonnes = ['Nom', 'Nom url', 'Categorie(s)', 'Date de mise a jour', 'Taille', 'Version Actuelle', 'Version IOS', 'Note', 'Date', 'Heure', 'URL Application', 'Pays', 'Categorie Principale Application','Categorie Secondaire Application']
 #dataFrameTmp : Data Frame Temporaire
dataFramTmp = pd.DataFrame(data = values , columns = colonnes) 
dataFramTmp = dataFramTmp.iloc[1: , :]
dataFramTmp.columns
df = dataFramTmp

Affichage de la data frame brute 

In [ ]:
df.head()

,Nom,Nom url,Categorie(s),Date de mise a jour,Taille,Version Actuelle,Version IOS,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application
1,Guide for WeChat Messenger,1045957209,Reference,"oct. 10, 2015",9.6 Mo,1,6,3.5,2022-05-20,11:06:46,https://apptica.com/charts/guide-for-wechat-me...,India,,Reference
2,Toca Kitchen Monsters,510301841,Education,"Mar 2, 2018",64.6 Mo,1.0.6,6,3.6,2022-05-20,10:54:34,https://apptica.com/charts/toca-kitchen-monste...,France,Kids,Education
3,Estimation Calculator,764708071,Games,"Dec 19, 2014",5.3 Mo,2.2,6,3.6,2022-05-20,11:27:07,https://apptica.com/charts/estimation-calculat...,Egypt,Games,Card
4,Kite Fighting,1008857841,Games,"oct. 14, 2016",22.6 Mo,3.3,6,3.7,2022-05-20,11:16:19,https://apptica.com/charts/kite-fighting/90731...,Brazil,Games,Entertainment
5,Transformers: Robots in Disguise,945490355,Games,"Feb 14, 2017",292.6 Mo,1.9,6,3.8,2022-05-20,10:55:04,https://apptica.com/charts/transformers-robots...,France,Kids,Entertainment


# Nettoyage de Donnes

d'apres la premiere observation on deduit que il y en a nombreux applications qui n'ont pas une categorie principale 

In [ ]:
tempdf = df 
## changer la veleur de vide vers nan
tempdf['Categorie Principale Application'] = tempdf['Categorie Principale Application'].replace(r'^\s*$', np.nan, regex=True)

## changer la valeur de nan a 0 pour faire la verification sur la valeur du la cellule 
tempdf.fillna({'Categorie Principale Application' : 0},  inplace=True)

column_categorie_princip = tempdf['Categorie Principale Application']    

for value in column_categorie_princip :
    if value == 0:
        tempdf['Categorie Principale Application'] = tempdf['Categorie(s)']

df = tempdf

### Remplacement des valeurs des cellules vide par des Nan 

In [ ]:
df_empty_fill = df.replace(r'^\s*$', np.nan, regex=True)
df = df_empty_fill

### Isolation des Applications qui contiennent des valeurs manquantes

In [ ]:
missing_valuesdf = df[df.isnull().any(1)]
missing_valuesdf

,Nom,Nom url,Categorie(s),Date de mise a jour,Taille,Version Actuelle,Version IOS,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application
130,Les défis du Professeur Muchi,1023760625,Education,"Sep 29, 2017",171.2 Mo,2.2.2002,8,NaN,2022-05-20,10:54:51,https://apptica.com/charts/les-defis-du-profes...,France,Education,Education
131,"YaduSurf : Vagues, Vent, Spots",510034859,Weather,"Aug 23, 2020",64.3 Mo,2.6.2005,8,NaN,2022-05-20,10:57:51,https://apptica.com/charts/yadusurf-vagues-ven...,France,Weather,Weather
132,Ticket Sports,1601902207,Sports,"May 17, 2022",133.8 Mo,1.4.2000,8,NaN,2022-05-20,11:14:20,https://apptica.com/charts/ticket-sports/90903...,Brazil,Sports,Sports
419,New Year & Christmas Editor,1544106817,Stickers,"Dec 18, 2020",247.3 Mo,1.0.1,9,NaN,2022-05-20,10:59:47,https://apptica.com/charts/new-year-christmas-...,France,Stickers,Celebrations
420,宝图盒子–表情包管理工‪具‬,1587833361,Stickers,"May 19, 2022",28.4 Mo,1.3.2000,9,NaN,2022-05-20,10:59:50,https://apptica.com/charts/application/8639924...,France,Stickers,Comics & Cartoons
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3577,Aygaz,1080944677,Lifestyle,"May 27, 2021",96.4 Mo,2.35,9.3,NaN,2022-05-20,11:34:04,https://apptica.com/charts/aygaz/962881/tr?cou...,Turkey,Lifestyle,Lifestyle
3610,Seif Pharmacy Online App,1579675841,Medical,"Mar 9, 2022",202.2 Mo,1.2,10.3,NaN,2022-05-20,11:19:10,https://apptica.com/charts/seif-pharmacy-onlin...,Egypt,Medical,Medical
3635,OYAK Mobil,1564032968,Finance,"Apr 29, 2022",42.3 Mo,3.1.2001,12.3,NaN,2022-05-20,11:33:25,https://apptica.com/charts/oyak-mobil/8048764/...,Turkey,Finance,Finance
3652,Holy Owly English for children,1189063739,Games,"May 17, 2022",706.9 Mo,2.6.2009,11.4,NaN,2022-05-20,10:54:28,https://apptica.com/charts/holy-owly-english-f...,France,Games,Education


Sauvgarde des valeurs manquantes en csv

In [ ]:
from google.colab import files
missing_valuesdf.to_csv('data_containes_missing_values_ios.csv', index = False) 
files.download('data_containes_missing_values_ios.csv')

### Les applications qui ne contiennent pas des valeurs manquantes

In [ ]:
droped_missing_values = df.dropna()
df = droped_missing_values

### Affichage de la data frame qui ne contient aucune valeurs manquante dans ces cellules

In [ ]:
df.head()

,Nom,Nom url,Categorie(s),Date de mise a jour,Taille,Version Actuelle,Version IOS,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application
1,Guide for WeChat Messenger,1045957209,Reference,"oct. 10, 2015",9.6 Mo,1,6,3.5,2022-05-20,11:06:46,https://apptica.com/charts/guide-for-wechat-me...,India,Reference,Reference
2,Toca Kitchen Monsters,510301841,Education,"Mar 2, 2018",64.6 Mo,1.0.6,6,3.6,2022-05-20,10:54:34,https://apptica.com/charts/toca-kitchen-monste...,France,Education,Education
3,Estimation Calculator,764708071,Games,"Dec 19, 2014",5.3 Mo,2.2,6,3.6,2022-05-20,11:27:07,https://apptica.com/charts/estimation-calculat...,Egypt,Games,Card
4,Kite Fighting,1008857841,Games,"oct. 14, 2016",22.6 Mo,3.3,6,3.7,2022-05-20,11:16:19,https://apptica.com/charts/kite-fighting/90731...,Brazil,Games,Entertainment
5,Transformers: Robots in Disguise,945490355,Games,"Feb 14, 2017",292.6 Mo,1.9,6,3.8,2022-05-20,10:55:04,https://apptica.com/charts/transformers-robots...,France,Games,Entertainment


### Nettoyage de la colonne Version Actuelles

+ Enlever les espaces 
+ Remplacer la valeur de certaines cellules par '.' 
+ Enlever les ponts doubles et tribles   
+ Enlever les caracteres speciaux et les lettres alphabetiques
+ Enlever les points au dernier 

* Enlever les espaces

In [ ]:
space_removing = df
space_removing['Version Actuelle'] = space_removing['Version Actuelle'].str.strip()
df = space_removing

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


* Remplacement de - | + _  par un point.

In [ ]:
special_cara = '+-_'
for special_caracter in special_cara:
  df['Version Actuelle'] = df['Version Actuelle'].str.replace(special_caracter,".", regex = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


* Enlever les lettre alphabetiques et les caracteres speciaux

In [ ]:
alphabet_and_special_caracters = '!#$%^&*@()abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
for spcial_and_caract in alphabet_and_special_caracters:
  df['Version Actuelle'] = df['Version Actuelle'].str.replace(spcial_and_caract,"", regex = True)

df['Version Actuelle'] = df['Version Actuelle'].str.replace(' ',"", regex = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### Typage 

In [ ]:
df.columns

Index(['Nom', 'Nom url', 'Categorie(s)', 'Date de mise a jour', 'Taille',
       'Version Actuelle', 'Version IOS', 'Note', 'Date', 'Heure',
       'URL Application', 'Pays', 'Categorie Principale Application',
       'Categorie Secondaire Application'],
      dtype='object')

In [ ]:
# df['Nom'] = df['Nom'].astype(object)
# df['Nom url'] = df['Nom url'].astype(object)
# df['Categorie(s)'] = df['Categorie(s)'].astype(object)
# df['Note'] = df['Note'].astype(float)
# df['Date'] = df['Date'].astype('datetime64[ns]')
# df['Heure'] = df['Heure'].astype('datetime64[ns]')
# df['URL Application'] = df['URL Application'].astype(object)
# df['Pays'] = df['Pays'].astype(object)
# df['Categorie Principale Application'] = df['Categorie Principale Application'].astype(object)
# df['Categorie Secondaire Application'] = df['Categorie Secondaire Application'].astype(object)
df

,Nom,Nom url,Categorie(s),Date de mise a jour,Taille,Version Actuelle,Version IOS,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application
1,Guide for WeChat Messenger,1045957209,Reference,"oct. 10, 2015",9.6 Mo,1,6,3.5,2022-05-20,11:06:46,https://apptica.com/charts/guide-for-wechat-me...,India,Reference,Reference
2,Toca Kitchen Monsters,510301841,Education,"Mar 2, 2018",64.6 Mo,1.0.6,6,3.6,2022-05-20,10:54:34,https://apptica.com/charts/toca-kitchen-monste...,France,Education,Education
3,Estimation Calculator,764708071,Games,"Dec 19, 2014",5.3 Mo,2.2,6,3.6,2022-05-20,11:27:07,https://apptica.com/charts/estimation-calculat...,Egypt,Games,Card
4,Kite Fighting,1008857841,Games,"oct. 14, 2016",22.6 Mo,3.3,6,3.7,2022-05-20,11:16:19,https://apptica.com/charts/kite-fighting/90731...,Brazil,Games,Entertainment
5,Transformers: Robots in Disguise,945490355,Games,"Feb 14, 2017",292.6 Mo,1.9,6,3.8,2022-05-20,10:55:04,https://apptica.com/charts/transformers-robots...,France,Games,Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3715,Uber - Request a ride,368677368,Travel,"May 16, 2022",373 Mo,3.510.10000,13.7,4.6,2022-05-20,10:57:17,https://apptica.com/charts/uber-request-a-ride...,France,Travel,Travel
3716,Uber Eats: Food Delivery,1058959277,Food & Drink,"May 16, 2022",309.6 Mo,6.112.10001,13.7,4.8,2022-05-20,10:55:35,https://apptica.com/charts/uber-eats-food-deli...,France,Food & Drink,Food & Drink
3717,Postmates - Food Delivery,512393983,Food & Drink,"May 16, 2022",291.7 Mo,6.112.10001,13.7,4.8,2022-05-20,11:00:57,https://apptica.com/charts/postmates-food-deli...,United States,Food & Drink,Food & Drink
3718,DeepL Translate,1552407475,Reference,"May 17, 2022",35 Mo,1.8.2001,14.7,4.8,2022-05-20,10:56:45,https://apptica.com/charts/deepl-translate/804...,France,Reference,Reference


### Creation d'une nouvelle colonne  ( Version Majeur )

* Typage de la colone Version Android:


Le typage de la colonne Version IOS en Float nous permet de faire des conditions pour pouvoir faire les categoreries les versions majeurs a la suite.

In [ ]:
df['Version IOS'] = df['Version IOS'].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


* Les condition pour regrouper les application en fonction de leur version android:

Le regroupement des applications en 8 versions majeures

In [ ]:
condition_versions_majeures = [( df['Version IOS'] >= 6.0 ) & ( df['Version IOS'] < 7.0 ),
                            ( df['Version IOS'] >= 7.0 ) & ( df['Version IOS'] < 8.0 ),
                            ( df['Version IOS'] >= 8.0 ) & ( df['Version IOS'] < 9.0 ),
                            ( df['Version IOS'] >= 9.0 ) & ( df['Version IOS'] < 10.0 ), 
                            ( df['Version IOS'] >= 10.0 ) & ( df['Version IOS'] < 11.0 ),
                            ( df['Version IOS'] >= 11.0 ) & ( df['Version IOS'] < 12.0 ),
                            ( df['Version IOS'] >= 12.0 ) & ( df['Version IOS'] < 13.0 ),
                            ( df['Version IOS'] >= 13.0 ) & ( df['Version IOS'] < 14.0 ),
                            ( df['Version IOS'] >= 14.0 ) & ( df['Version IOS'] < 15.0 ),
                            ( df['Version IOS'] >= 15.0 ) & ( df['Version IOS'] < 16.0 )
                            ]
versions_majeures = [ '6','7','8','9','10','11','12','13','14','15']

* Creation de la colonne Version Majeure

In [ ]:
df['Version Majeur'] = np.select(condition_versions_majeures,versions_majeures)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


* Affichage de nouvelle DataFram contentante la colonne Version Majeur

In [ ]:
df = df.reset_index(drop=True)
df.head()

,Nom,Nom url,Categorie(s),Date de mise a jour,Taille,Version Actuelle,Version IOS,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application,Version Majeur
0,Guide for WeChat Messenger,1045957209,Reference,"oct. 10, 2015",9.6 Mo,1,6.0,3.5,2022-05-20,11:06:46,https://apptica.com/charts/guide-for-wechat-me...,India,Reference,Reference,6
1,Toca Kitchen Monsters,510301841,Education,"Mar 2, 2018",64.6 Mo,1.0.6,6.0,3.6,2022-05-20,10:54:34,https://apptica.com/charts/toca-kitchen-monste...,France,Education,Education,6
2,Estimation Calculator,764708071,Games,"Dec 19, 2014",5.3 Mo,2.2,6.0,3.6,2022-05-20,11:27:07,https://apptica.com/charts/estimation-calculat...,Egypt,Games,Card,6
3,Kite Fighting,1008857841,Games,"oct. 14, 2016",22.6 Mo,3.3,6.0,3.7,2022-05-20,11:16:19,https://apptica.com/charts/kite-fighting/90731...,Brazil,Games,Entertainment,6
4,Transformers: Robots in Disguise,945490355,Games,"Feb 14, 2017",292.6 Mo,1.9,6.0,3.8,2022-05-20,10:55:04,https://apptica.com/charts/transformers-robots...,France,Games,Entertainment,6


* Affichage des colonnes de la DataFrame

In [ ]:
df.columns

Index(['Nom', 'Nom url', 'Categorie(s)', 'Date de mise a jour', 'Taille',
       'Version Actuelle', 'Version IOS', 'Note', 'Date', 'Heure',
       'URL Application', 'Pays', 'Categorie Principale Application',
       'Categorie Secondaire Application', 'Version Majeur'],
      dtype='object')

On voit que les colonnes ne sont pas en ordre 

* Ordonnancement des colonnes.

In [ ]:
column_names = ['Nom', 'Nom url', 'Categorie(s)', 'Date de mise a jour', 'Taille', 'Version Actuelle', 'Version Majeur','Version IOS', 'Note', 'Date', 'Heure', 'URL Application', 'Pays', 'Categorie Principale Application','Categorie Secondaire Application']
df = df.reindex(columns=column_names)
df = pd.DataFrame(df)

# Resultat Finale

In [ ]:
df.head()

,Nom,Nom url,Categorie(s),Date de mise a jour,Taille,Version Actuelle,Version Majeur,Version IOS,Note,Date,Heure,URL Application,Pays,Categorie Principale Application,Categorie Secondaire Application
0,TV - Télévision Française ‪!‬,968729950,News,"nov. 5, 2019",20.9 Mo,2,10,10.0,1.0,2022-05-20,2022-06-09 10:56:37,https://apptica.com/charts/tv-television-franc...,France,News,News
1,ZestMoney,1597244251,Finance,"May 12, 2022",59.9 Mo,1.0.7,12,12.0,1.0,2022-05-20,2022-06-09 11:05:00,https://apptica.com/charts/zestmoney/9090601/i...,India,Finance,Finance
2,Manipal Hospitals,1524836456,Medical,"nov. 12, 2021",60.8 Mo,1.26,12,12.0,1.0,2022-05-20,2022-06-09 11:05:38,https://apptica.com/charts/manipal-hospitals/7...,India,Medical,Medical
3,RTO Vehicle Registration Info,1474673684,Reference,"Aug 1, 2021",25 Mo,1.4,11,11.0,1.0,2022-05-20,2022-06-09 11:06:38,https://apptica.com/charts/rto-vehicle-registr...,India,Reference,Reference
4,Meu Imposto de Renda,922529225,Business,"Apr 11, 2022",33 Mo,8.4.2000,15,15.0,1.0,2022-05-20,2022-06-09 11:11:21,https://apptica.com/charts/meu-imposto-de-rend...,Brazil,Business,Business


Sauvgarde de la Dataframe nettoyée

In [ ]:
from google.colab import files
df.to_csv('Cleaned_Data_IOS.csv', index = False) 
files.download('Cleaned_Data_IOS.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>